In [ ]:
###insan sınıflandırma

import numpy as np
from skimage import transform #görüntüleri yeniden boyutlandırma kaydırma döndürme
from skimage.transform import pyramid_gaussian #görüntü piramitleri oluşturur
from imutils.object_detection import non_max_suppression #bounding box, birbirine çok yakın kutuları birleştiri
import imutils #image için utils (yardımcı fonklar)
from skimage.feature import hog #özellik çıkarımı, şekil ve kenar bilgisi
from joblib import load #eğitilmiş modeli kaydetmek ve yüklemek. (.pkl)
import cv2
from skimage import color #renk uzayında dönüşümler
import matplotlib.pyplot as plt
import os
import glob #belirli bir klasördeki ddosyaları listleemk için kullanılır

pos_im_path = "pos_person" #insan olan görüntüler
neg_im_path = "neg_person" #insan olmayan görüntüler
min_wdw_sz = [64,128] #bounding box boyutu
step_size = [10,10] #bounding box kaç piksel sağ sol aşağı yukarı kaysın
orientations = 9 #bin size 10,30,50,...170
pixels_per_cell = [6,6] #hog için görüntü içerisinde seçileceği piksel, genelde (10,10) ama daha hassas çalışmak istiyoruz.
cells_per_block = [2,2] #blok normalizasyonu için bloğun size
#visualize = False #model eğitilirken görselleştime olmasın
pos_feat_ph = "pos" #çıktısı pozitif olanlar bu klasöre
neg_feat_ph = "neg" #negatif olanlar bu klasöre kaydedilsin
model_path = "models" #model bu yola kaydedilsin
threshold = .3 #0.3 den büyük ise insan küçük ise değil

def sliding_window(image, window_size, step_size):
  for y in range(0, image.shape[0],step_size[1]):
    for x in range(0, image.shape[1],step_size[0]):
      yield (x,y,image[y: y + window_size[1], x:x+window_size[0]])
      #görüntüyü küçük pencerelere bölüyor. her bir adımda pencere 10 adım kaydırılıyor. her durumda 64*128'lik bir pencere oluşturuluyor
      #yield her pencereyi sırayla veriyoor

def detector(filename):
  im = cv2.imread(filename)
  im = imutils.resize(im, width = min(400, im.shape[1])) #400*y lik görüntüye yeniden ölçekle
  min_wdw_sz = (64, 128)
  step_size = (10,10)
  downscale = 1.6 #piramit için küçültme oranı

  clf = joblib.load(os.path.jon(model_path, 'svm.modlel')) #daha önce eğtilmiş svm modeli çağırıyor

  detections = []

  scale = 0
  """piramitin içine koyduğumuz şekil scale'in üssel azalmasıyla
  önce gray yapılır sonra hog algoritmasına sokulur
  sonra yeniden ölçeklendirilir (?)
  en sonunda svm algoritmasına verilir"""
  for im_scaled in pyramid_gaussian(im, downscale = downscale):
    if im_scaled.shape[0] < min_wdw_sz[1] or im_scaled.shape[1] < min_wdw_sz[0]:
      break
    for (x,y,im_window) in sliding_window(im_scaled, min_wdw_sz, step_size):
      if im_window.shape[0] != min_wdw_sz[1] or im_window.shape[1]!=min_wdw_sz[0]:
        continue
      im_window = color.rgb2gray(im_window)
      fd = hog(im_window, orientations = 9, picels_per_cell=(6,6), cells_per_block=(2,2), block_norm='L1',
               visualize = False,
               transform_sqrt =False,
               feature_vector = True)
      #hogların sonucu genelde 1 boyutlu vektör olarak gelir.
      #decision_function ve predickt 2D array bekler(örnek sayısı, özellik sayısı)
      #(1,3670): tek örnek, 3670 özellik
      fd = fd.reshape(1,-1)
      pred = clf.predict(fd)

      if pred ==1: #insan dedi diyelim
       #decision_function: güven skoru of svö
        if clf.decision_function(fd) > 0.5:
          detections.append((int(x * (downscale**scale)), int(y * (downscale**scale)), clf.decision_function(fd), int(min_wdw_sz[0] * (downscale**scale)),int(min_wdw_sz[1] * (downscale**scale))))
    scale+=1
  clone = im.copy()

  rects = np.array([[x,y,x+w, y+h] for (x,y,_,w,h) in detections])
  #verideki her karenin en sol köşesi ile en alt sol köşesi alınıyor. (0,0) olsun mesela sağ alt (64,128 olur)
  #bu değer non_max_suppresion2ın bounding box yapmasını sağlayacak
  sc = [score[0] for (x,y,score,w,h) in detections] #her pencere içindeki skora bakıyor
  pick = non_max_suppression(rects, probs = sc, overlapTresh = 0.3) #bu skorlardan en yükseğini seçiyor.
  #mesela pencere 12in skoru: 0.9, p2 sc : 0.85, p3 sc: 0.6, pencere1'i seçti

  for (x_tl, y_tl,_,w,h) in detections:
    cv2.rectangle(im, (x_tl,y_tl), (x_tl+w, y_tl+h), (0,255,0), thickness = 2)
  for (xA, yA, xB, yB) in pick:
    cv2.rectangle(clone,(xA,yA),(xB,yB),(0,255,0),2)
  scale_percent=70
  w=clone.shape[1]
  h=clone.shape[0]
  width = int(w * scale_percent / 100)
  height = int(h * scale_percent / 100)
  dim = (width, height)
  resized = cv2.resize(
          clone, dim, interpolation=cv2.INTER_AREA)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
  plt.title("Raw Detection before NMS")
  plt.show()

  plt.axis("off")
  plt.imshow(cv2.cvtColor(clone, cv2.COLOR_BGR2RGB))
  plt.title("Final Detections after applying NMS")
  plt.show()

import os
import glob
import cv2
# joblib, imutils, vb. diğer importlar kodun başında olmalı

# ----------------------------
# DÜZELTİLMİŞ process_folder
# ----------------------------

def process_folder(folder_path):
    """
    Verilen klasördeki tüm görüntü dosyalarını sırayla işler.

    detector(filename) fonksiyonu zaten yükleme, tespit, NMS ve görselleştirmeyi yapar.
    Bu fonksiyonun görevi sadece dosya yollarını yönetmektir.
    """

    # Desteklenen dosya uzantıları
    valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff')

    print(f"İşleniyor: {folder_path} klasöründeki tüm resimler.")

    # glob ile klasördeki tüm dosyaları gez
    # glob.iglob, büyük klasörler için daha verimlidir
    for file_path in glob.iglob(os.path.join(folder_path, '*.*')):

        # Sadece resim dosyalarını işle
        if file_path.lower().endswith(valid_extensions):
            print(f"-> Tespit başlatılıyor: {os.path.basename(file_path)}")

            try:
                # detector fonksiyonunu doğrudan dosya yolu ile çağır
                detector(file_path)
            except Exception as e:
                print(f"Hata oluştu ({os.path.basename(file_path)}): {e}")

    print("\n✅ Tüm resimler işlendi.")






In [19]:
# Kütüphane gereksinimleri
import glob
import os
from skimage.feature import hog
import numpy as np
import cv2
# Scikit-learn kütüphaneleri kullanılmadığı için kaldırılmıştır.

# ----------------------------------------------------
# 1. KONFİGÜRASYON
# ----------------------------------------------------
# Klasör yolları (Google Drive yolları, çalışması için Drive'ın bağlı olması gerekir)
pos_im_path = r'/content/drive/MyDrive/data_dog/1'  # Pozitif (köpek) görseller
neg_im_path = r'/content/drive/MyDrive/data_dog/0'  # Negatif (köpek olmayan) görseller

# HOG Parametreleri (Görsellerin bu parametrelere uygun boyutta olması beklenir, aksi halde hata verir)
HOG_PARAMS = {
    'orientations': 9,
    'pixels_per_cell': (6, 6),
    'cells_per_block': (2, 2),
    'block_norm': 'L1',
    'visualize' :False,
    'transform_sqrt': False,
    'feature_vector': True,
}


samples = []
labels = []

# ----------------------------------------------------
# 2. ÖZELLİK ÇIKARIMI (HOG)
# ----------------------------------------------------

# Pozitif Örnekleri Al (Varsayım: .png dosyaları)
print("Pozitif örnekler işleniyor...")
for filename in glob.glob(os.path.join(pos_im_path, '*.png')):
    img = cv2.imread(filename, 0) # Gri tonlamada oku
    if img is not None:
        hist = hog(img, **HOG_PARAMS)
        samples.append(hist)
        labels.append(1)

# Negatif Örnekleri Al (Varsayım: .jpg dosyaları)
print("Negatif örnekler işleniyor...")
for filename in glob.glob(os.path.join(neg_im_path, '*.jpg')):
    img = cv2.imread(filename, 0) # Gri tonlamada oku
    if img is not None:
        hist = hog(img, **HOG_PARAMS)
        samples.append(hist)
        labels.append(0)

# ----------------------------------------------------
# 3. VERİ ÖN İŞLEME VE KARIŞTIRMA
# ----------------------------------------------------

# Nesneleri Numpy Objelerine Çevir
samples = np.float32(samples)
labels = np.array(labels)

if len(samples) == 0:
    print("\n[HATA] Hiçbir görüntü yüklenmedi veya HOG özelliği çıkarılamadı. Dosya yollarını kontrol edin.")
else:
    # Örnekleri Karıştır
    rand = np.random.RandomState(321)
    shuffle = rand.permutation(len(samples))
    samples = samples[shuffle]
    labels = labels[shuffle]

    # ----------------------------------------------------
    # 4. SVM EĞİTİMİ (OpenCV ml)
    # ----------------------------------------------------

    # SVM Sınıflandırıcısını Oluştur ve Ayarla
    svm = cv2.ml.SVM_create()
    svm.setType(cv2.ml.SVM_C_SVC)

    # RBF yerine Lineer Kernel kullanmak daha yaygın ve hızlıdır, ancak kodunuzdaki RBF ayarlarını koruyoruz.
    svm.setKernel(cv2.ml.SVM_RBF) # Radyal Tabanlı Fonksiyon (RBF)
    svm.setGamma(5.383)
    svm.setC(2.67)

    print("\nSVM eğitimi başlatılıyor...")
    # Train
    # cv2.ml.ROW_SAMPLE: Her bir satırın bir örneği temsil ettiğini belirtir (Bizim verimizde bu geçerli)
    svm.train(samples, cv2.ml.ROW_SAMPLE, labels)
    print("Eğitim tamamlandı.")

    # Destek Vektörleri (Debug)
    # Sadece doğru yazım kullanılırsa çalışır:
    print("\nDestek Vektörleri:")
    print(svm.getSupportVectors())

    # Destek Vektörlerinin indislerini yazdırmak için:
    print("\nDestek Vektörlerinin İndisleri:")
    print(svm.getSupportVectorIndices())

    # Modeli Kaydet
    svm.save(r'/content/drive/MyDrive/svm_data.dat')
    print(f"\nModel başarıyla kaydedildi: /content/drive/MyDrive/svm_data.dat")

Pozitif örnekler işleniyor...
Negatif örnekler işleniyor...


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (48,) + inhomogeneous part.

In [ ]:
# ----------------------------
# Ana Program
# ----------------------------
if __name__ == "__main__":
    svm_model = joblib.load(MODEL_PATH)
    TEST_FOLDER = r'C:\Users\Richa Agrawal\Desktop\computer vision\video\re-svm\object_detector\test_image'
    process_folder(TEST_FOLDER, svm_model)


In [22]:
import cv2
import numpy as np
import os
import glob
from skimage.feature import hog
import random
from sklearn.utils import shuffle

# ==============================================================================
# 1. SABİT TANIMLAMALARI
# ==============================================================================

# HOG ve SVM için temel sabitler (Bu değerleri projenize göre ayarlayabilirsiniz)
MIN_WDW_SZ = (64, 128) # HOG pencere boyutu (genişlik, yükseklik)
PPC = (8, 8)           # pixels_per_cell
CPB = (2, 2)           # cells_per_block
ORIENTATIONS = 9       # Yönlendirme sayısı

# HOG Parametre Sözlüğü
HOG_PARAMS = {
    'orientations': ORIENTATIONS,
    'pixels_per_cell': PPC,
    'cells_per_block': CPB,
    'visualize': False,
    'channel_axis': None # Gri tonlama (tek kanal) kullanıldığı için None
}

# Veri ve Model Yolları (Kendi klasör yollarınıza göre düzenleyin)
POS_IM_PATH = '/content/drive/MyDrive/data_dog/1'  # Pozitif örneklerin yolu (örneğin kedi resimleri)
NEG_IM_PATH = '/content/drive/MyDrive/data_dog/0'  # Negatif örneklerin yolu (örneğin arka plan resimleri)
MODEL_SAVE_PATH = '/content/drive/MyDrive/svm_model_dog.xml'

# ==============================================================================
# 2. EĞİTİM FONKSİYONU
# ==============================================================================

def train_svm():
    """HOG özelliklerini çıkarır, veriyi karıştırır ve OpenCV SVM modelini eğitip kaydeder."""
    samples = []
    labels = []

    img_extensions = ['*.png', '*.jpg', '*.jpeg']

    # --- Beklenen HOG Vektör Uzunluğunu Hesaplama ---
    # Bu, tüm vektörlerin aynı boyutta olması için bir referans noktasıdır.
    try:
        # Örnek bir 64x128 siyah görüntüden HOG vektör uzunluğunu hesapla
        temp_img = np.zeros(MIN_WDW_SZ, dtype=np.uint8)
        # HOG'u yalnızca bir kere hesaplayıp uzunluğunu alıyoruz
        expected_len = len(hog(temp_img, **HOG_PARAMS))
        print(f"[DEBUG] Beklenen HOG Vektör Uzunluğu: {expected_len}")
    except Exception as e:
        print(f"[HATA] HOG Parametreleri veya MIN_WDW_SZ hatalı. Eğitim durduruluyor. Hata: {e}")
        return False


    # --- Pozitif Örnekler İşleniyor (Etiket: 1) ---
    print("\nPozitif örnekler işleniyor...")
    for ext in img_extensions:
        for filename in glob.glob(os.path.join(POS_IM_PATH, ext)):
            img = cv2.imread(filename, 0) # Görüntüyü gri tonlamalı oku
            if img is not None:
                # Görüntüyü HOG pencere boyutuna yeniden boyutlandır
                img = cv2.resize(img, MIN_WDW_SZ)
                try:
                    hist = hog(img, **HOG_PARAMS)

                    # KRİTİK KONTROL: Vektör uzunluğunu kontrol et
                    if len(hist) == expected_len:
                        samples.append(hist)
                        labels.append(1)
                    else:
                        print(f"[UYARI] Pozitif Dosya Hatalı: {os.path.basename(filename)} - Beklenmeyen HOG boyutu ({len(hist)}). Atlanıyor.")
                except Exception as e:
                    print(f"[UYARI] Pozitif İşleme Hatası: {os.path.basename(filename)} atlanıyor. Hata: {e}")


    # --- Negatif Örnekler İşleniyor (Etiket: 0) ---
    print("\nNegatif örnekler işleniyor...")
    for ext in img_extensions:
        for filename in glob.glob(os.path.join(NEG_IM_PATH, ext)):
            img = cv2.imread(filename, 0) # Görüntüyü gri tonlamalı oku
            if img is not None:
                # Görüntüyü HOG pencere boyutuna yeniden boyutlandır
                img = cv2.resize(img, MIN_WDW_SZ)
                try:
                    hist = hog(img, **HOG_PARAMS)

                    # KRİTİK KONTROL: Vektör uzunluğunu kontrol et
                    if len(hist) == expected_len:
                        samples.append(hist)
                        labels.append(0)
                    else:
                        print(f"[UYARI] Negatif Dosya Hatalı: {os.path.basename(filename)} - Beklenmeyen HOG boyutu ({len(hist)}). Atlanıyor.")
                except Exception as e:
                    print(f"[UYARI] Negatif İşleme Hatası: {os.path.basename(filename)} atlanıyor. Hata: {e}")


    # --- Veri Ön Kontrolleri ---
    if len(samples) == 0:
        print("\n[HATA] Eğitim için hiçbir görüntü yüklenmedi. Yolları ve uzantıları kontrol edin.")
        return False

    # Sınıf sayısı kontrolü
    pos_count = labels.count(1)
    neg_count = labels.count(0)
    if pos_count == 0 or neg_count == 0:
        print("\n[HATA] SVM eğitimi için her iki sınıftan da (pozitif ve negatif) örneklere ihtiyaç vardır.")
        print(f"[DEBUG] Toplanan Pozitif Örnek: {pos_count}, Negatif Örnek: {neg_count}")
        return False

    # --- NumPy'a Çevirme ve Karıştırma ---
    print(f"\nToplam {len(samples)} örnek ile eğitime hazırlanılıyor.")

    # ValueError'a neden olan satır: Şimdi samples listesindeki tüm HOG vektörleri
    # aynı uzunlukta olduğu için hata oluşmayacak.
    samples = np.float32(samples)
    labels = np.array(labels)

    # Veriyi Karıştır (Genellikle daha iyi genelleme için yapılır)
    samples, labels = shuffle(samples, labels, random_state=42)

    # --- SVM Eğitimi ---
    svm = cv2.ml.SVM_create()
    svm.setType(cv2.ml.SVM_C_SVC) # Sınıflandırma için C-SVC
    svm.setKernel(cv2.ml.SVM_RBF) # Radial Basis Function (RBF) çekirdeği
    svm.setGamma(5.383)           # Bu parametreler optimizasyonla bulunmuştur
    svm.setC(2.67)

    print("SVM eğitimi başlatılıyor...")
    # cv2.ml.ROW_SAMPLE: Her satırın bir veri örneği olduğunu belirtir.
    svm.train(samples, cv2.ml.ROW_SAMPLE, labels)
    print("Eğitim tamamlandı.")

    # Modeli Kaydet
    svm.save(MODEL_SAVE_PATH)
    print(f"\nModel başarıyla kaydedildi: {MODEL_SAVE_PATH}")
    return True

# ==============================================================================
# 3. FONKSİYON ÇAĞRISI (Test için)
# ==============================================================================

if __name__ == '__main__':
    train_svm()

[DEBUG] Beklenen HOG Vektör Uzunluğu: 3780

Pozitif örnekler işleniyor...

Negatif örnekler işleniyor...

Toplam 99 örnek ile eğitime hazırlanılıyor.
SVM eğitimi başlatılıyor...
Eğitim tamamlandı.

Model başarıyla kaydedildi: /content/drive/MyDrive/svm_model_dog.xml


In [23]:
import numpy as np
import cv2
import os
import glob
from skimage.feature import hog
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder

# ====================================================
# I. KONFİGÜRASYON VE PARAMETRELER
# ====================================================

# --- TEST YOLLARI ---
# Kaydedilmiş modelinizin yolu (Önceki kodda MODEL_SAVE_PATH'a eşittir)
MODEL_PATH = r'/content/drive/MyDrive/svm_model_dog.xml'
# Test verilerinizin ana klasörü (altında 1 ve 0 klasörleri olmalı)
TEST_DATA_DIR = r'/content/drive/MyDrive/data_dog_test'

# --- HOG PARAMETRELERİ (Eğitimde kullanılanlarla AYNI olmalıdır) ---
MIN_WDW_SZ = (64, 128) # HOG pencere boyutu
HOG_PARAMS = {
    'orientations': 9,
    'pixels_per_cell': (8, 8),
    'cells_per_block': (2, 2),
    'visualize': False,
    'channel_axis': None
    # Not: Eğitimde kullandığınız HOG_PARAMS sözlüğünü buraya tamamen aktarın.
}

# ====================================================
# II. VERİ YÜKLEME VE ÖZELLİK ÇIKARMA
# ====================================================

def load_test_data(data_dir, target_size, hog_params):
    """Test verilerini yükler, HOG özelliklerini çıkarır ve etiketler."""
    test_features = []
    true_labels = []

    img_extensions = ['*.png', '*.jpg', '*.jpeg']

    # 1. Beklenen HOG Vektör Uzunluğunu Hesapla (Hata Kontrolü İçin)
    temp_img = np.zeros(target_size, dtype=np.uint8)
    expected_len = len(hog(temp_img, **hog_params))

    print(f"Test verileri işleniyor: {data_dir}")

    # data_dog_test/1 ve data_dog_test/0 alt klasörlerini tara
    for class_name in ['1', '0']:
        class_path = os.path.join(data_dir, class_name)
        label_value = int(class_name) # 1 veya 0

        print(f"-> Sınıf {class_name} işleniyor (Etiket: {label_value})...")

        for ext in img_extensions:
            for filename in glob.glob(os.path.join(class_path, ext)):
                img = cv2.imread(filename, 0)
                if img is not None:
                    img = cv2.resize(img, target_size)
                    try:
                        features = hog(img, **hog_params)

                        if len(features) == expected_len:
                            test_features.append(features)
                            true_labels.append(label_value)
                        # else: Hatalı vektörler atlanmıştır.

                    except Exception as e:
                        print(f"HOG çıkarma hatası ({os.path.basename(filename)}): {e}. Atlanıyor.")

    return np.float32(test_features), np.array(true_labels)


# ====================================================
# III. MODEL YÜKLEME VE TAHMİN
# ====================================================

def test_model(X_test, y_test, model_path):
    """Kaydedilmiş SVM modelini yükler ve tahmin yapar."""

    # 1. Modeli Yükle
    try:
        clf = cv2.ml.SVM_load(model_path)
    except Exception as e:
        print(f"\n[FATAL HATA] Model yüklenemedi: {model_path}. Yolu ve dosya adını kontrol edin.")
        print(f"Hata detayı: {e}")
        return

    # 2. Tahmin Yap
    print("\nTahminler başlatılıyor...")

    # predict metodu, Raw Output (Güven Skoru) döndürür.
    # StatModel_RAW_OUTPUT bayrağı ile karar fonksiyonu çıktısını alırız.
    _, pred_raw = clf.predict(X_test, flags=cv2.ml.StatModel_RAW_OUTPUT)

    # Raw output genellikle pozitif değerler için pozitif, negatif değerler için negatif işaretlidir.
    # Bizim etiketlerimiz 1 ve 0 olduğu için, skoru etiketlere dönüştürmeliyiz:
    # Skor > 0 (veya CONFIDENCE_THRESHOLD) ise 1 (pozitif sınıf), aksi takdirde 0.

    # Not: Eğitimde kullandığınız confidence threshold (0.5) yerine
    # basitçe işaret (sign) kontrolü yapalım, bu SVM'de daha yaygındır.
    # Yani Karar Sınırı (Decision Boundary) 0'dır.
    y_pred = np.where(pred_raw.flatten() > 0, 1, 0)

    # 3. Metrikleri Hesapla
    print("Metrikler hesaplanıyor...")

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro', zero_division=0)
    recall = recall_score(y_test, y_pred, average='macro', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='macro', zero_division=0)

    # 4. Sonuçları Yazdır
    print("\n==============================================")
    print("📊 SVM Test Sonuçları (HOG Özellikleri):")
    print("==============================================")
    print(f"Toplam Test Örneği: {len(y_test)}")
    print(f"✅ Doğruluk (Accuracy): {accuracy:.4f}")
    print(f"✅ Hassasiyet (Precision): {precision:.4f}")
    print(f"✅ Geri Çağırma (Recall): {recall:.4f}")
    print(f"✅ F1-Skoru: {f1:.4f}")
    print("==============================================")


# ====================================================
# IV. ANA ÇALIŞTIRMA BLOĞU
# ====================================================

if __name__ == '__main__':
    # 1. Veriyi Yükle ve HOG Özelliklerini Çıkar
    X_test, y_test = load_test_data(TEST_DATA_DIR, MIN_WDW_SZ, HOG_PARAMS)

    if len(X_test) > 0:
        # 2. Modeli Test Et ve Metrikleri Göster
        test_model(X_test, y_test, MODEL_PATH)
    else:
        print("\n[UYARI] Test verisi bulunamadı veya işlenemedi. Lütfen klasör yollarını kontrol edin.")

Test verileri işleniyor: /content/drive/MyDrive/data_dog_test
-> Sınıf 1 işleniyor (Etiket: 1)...
-> Sınıf 0 işleniyor (Etiket: 0)...

Tahminler başlatılıyor...
Metrikler hesaplanıyor...

📊 SVM Test Sonuçları (HOG Özellikleri):
Toplam Test Örneği: 100
✅ Doğruluk (Accuracy): 0.5000
✅ Hassasiyet (Precision): 0.2500
✅ Geri Çağırma (Recall): 0.5000
✅ F1-Skoru: 0.3333
